# Import packages

In [5]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

# Import  and merge data from oecd-files 

In [9]:
# Import OECD-data from CSV-files
oecd_data1 = pd.read_csv('Data/CITIES_POPULATION.csv', sep="|", header=0)
oecd_data2 = pd.read_csv('Data/CITIES_AGE.csv', sep="|", header=0)
oecd_data3 = pd.read_csv('Data/CITIES_ECONOMY.csv', sep="|", header=0)
oecd_data4 = pd.read_csv('Data/CITIES_LABOUR.csv', sep="|", header=0)
oecd_data5 = pd.read_csv('Data/CITIES_TERRITORY.csv', sep="|", header=0)
oecd_data6 = pd.read_csv('DATA/CITIES_ENVIRONMENT.csv', sep="|", header=0)

In [15]:
# Put all OECD-data into dataframe and append all dataframes
oecd_data_df = [oecd_data1, oecd_data2, oecd_data3, oecd_data4, oecd_data5, oecd_data6]
# Call concat method
oecd_df = pd.concat(oecd_data_df)

In [13]:
# TEMPORARY: visualize the results
oecd_df

## Rename columnames

In [16]:
# create a dictionary where key = old name and value = new name
dict = {'METRO_ID': 'metroId',
        'Metropolitan areas': 'metropolitanAreas',
        'VAR': 'var',
       'Variables' : 'variables',
        'TIME' : 'time',
        'Year' : 'year',
        'Unit Code': 'unitCode',
        'Unit' : 'unit',
        'PowerCode Code' : 'powerCodeCode',
        'PowerCode': 'powerCode',
        'Reference Period Code' : 'referencePeriodCode',
        'Reference Period' : 'referencePeriod',
        'Value':'value',
        'Flag Codes' : 'flagCodes',
        'Flags': 'flags'
       }
  
# call rename () method
oecd_df.rename(columns=dict,
          inplace=True)

In [17]:
# Checking the data
print(oecd_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586301 entries, 0 to 110607
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   metroId              586301 non-null  object 
 1   metropolitanAreas    586301 non-null  object 
 2   var                  586301 non-null  object 
 3   variables            586301 non-null  object 
 4   time                 586301 non-null  int64  
 5   year                 586301 non-null  int64  
 6   unitCode             138868 non-null  object 
 7   unit                 138868 non-null  object 
 8   powerCodeCode        586301 non-null  int64  
 9   powerCode            586301 non-null  object 
 10  referencePeriodCode  0 non-null       float64
 11  referencePeriod      0 non-null       float64
 12  value                584753 non-null  float64
 13  flagCodes            29450 non-null   object 
 14  flags                29450 non-null   object 
dtypes: float64(3), in

# Add longitude and latitude to join data
Strings that represent metropolitan areas are often different, which makes it difficult to join data from different sources. For example, "Lafayette (IN)" and "Lafayette, IN" and "Lafayette". Therefore, longitude and latitude are added as a unique identifier to join data.
## Create unique list of metropolitans from dataframe

In [18]:
# The length of the list is determined first, to determine which function to apply
len(oecd_df['metropolitanAreas'].unique().tolist())

665

In [19]:
# Assign the list to a variable
metro_list= oecd_df['metropolitanAreas'].unique().tolist()
# Convert the list into a dataframe
df_metropolitan=DataFrame (metro_list, columns=['metropolitanAreas'] )
# Check the dataframe
df_metropolitan

,metropolitanAreas
0,Allen
1,Warsaw
2,Kagoshima
3,Mexicali
4,Leicester
...,...
660,Cass
661,Carmen
662,Linn
663,Lafayette (IN)


## Add longitudes and latitudes to metropolitans

In [20]:
# declare an empty list to store latitude and longitude of values of the metropolitanAreas column
longitude = []
latitude = []

# use function to find the coordinate of a given metropolitanArea
def findGeocode(city):

    # try and catch is used to overcome the exception thrown by geolocator using geocodertimedout
    try:
        # Specify the user_agent as app name: this is my (Arabella) local address
        geolocator = Nominatim(user_agent="127.0.0.1:8888") 
        return geolocator.geocode(city)
    except GeocoderTimedOut:
        return findGeocode(city)

# each value from city column will be fetched and sent to function find_geocode
for i in (df_metropolitan["metropolitanAreas"]):

    if findGeocode(i) != None:

        loc = findGeocode(i)

        # coordinates returned from function is stored into two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

    # if coordinate for a city not found, insert "NaN" indicating missing value
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [21]:
# now add each column to dataframe
df_metropolitan["longitude"] = longitude
df_metropolitan["latitude"] = latitude

In [22]:
df_metropolitan

,metropolitanAreas,longitude,latitude
0,Allen,-95.317021,37.888217
1,Warsaw,21.006725,52.231958
2,Kagoshima,130.547408,31.521587
3,Mexicali,-115.448325,32.624862
4,Leicester,-1.133079,52.636140
...,...,...,...
660,Cass,-94.351037,33.056086
661,Carmen,125.674482,7.375394
662,Linn,-91.603793,42.063183
663,Lafayette (IN),-86.890423,40.403467


# Merge and write to csv

In [30]:
# merge the dataframes
df_oecdCoord = pd.merge(oecd_df, df_metropolitan, on="metropolitanAreas")

# write the dataframe to csv
df_oecdCoord.to_csv('Data/CITIES_ALL', header="True", sep="|", doublequote=True)
